# Creating a Static Dashboard for Risk Scoring and Alert Interpretation

### Imports:

In [ ]:
import os
import numpy as np
import pandas as pd

### Defining Anomaly Interpreter Class:

In [ ]:
class UEBAAnomalyInterpretor:
    """
    Transforms a raw anomaly score into interpretable UEBA risk outputs.
    """
    
    def __init__(self, high_risk_percentile: float=95.0, medium_risk_percentile: float=80.0) -> None:
        """
        Initializes the risk threshold.
        
        Args:
            high_risk_percentile: Percentile for high-risk alerts
            medium_risk_percentile: Percentile for medium-risk alerts
            
        Returns:
            None:
        """
        self.high_risk_percentile = high_risk_percentile
        self.medium_risk_percentile = medium_risk_percentile
        
    
    def normalize_scores(self, anomaly_scores: np.ndarray) -> np.ndarray:
        """
        Converts raw anomaly scores to percentile ranks.
        
        Args:
            anomaly_scores: The raw anomaly scores
            
        Returns:
            np.ndarray: Percentile-normalized anomaly scores (0-100)
        """
        norm_scores = pd.DataFrame(data=anomaly_scores).rank(pct=True).values * 100
        return norm_scores
    
    
    def assign_risk_level(self, percentile_scores: np.ndarray) -> list:
        """
        Assigns risk categories based on percentile thresholds.
        
        Args:
            percentile_scores: Percentile-normalized scores
            
        Returns:
            list: A list containing risk level labels
        """
        risk_levels = []
        
        # Assigning risk label according to percentile rank
        for score in percentile_scores:
            if score >= self.high_risk_percentile:
                risk_levels.append("HIGH")
            elif score >= self.medium_risk_percentile:
                risk_levels.append("MEDIUM")
            else:
                risk_levels.append("LOW")
                
        return risk_levels
    
    
    def build_analyst_table(self, metadata: pd.DataFrame, raw_scores: np.ndarray) -> pd.DataFrame:
        """
        Constructs an analyst-facing table with risk context.
        
        Args:
            metadata: User/time identifiers
            raw_scores: The raw anomaly scores
            
        Returns:
            pd.DataFrame: An interpretable anomaly table
        """
        # Calculating percentile and risk levels of raw scores
        percentiles = self.normalize_scores(raw_scores)
        risk_levels = self.assign_risk_level(percentiles)
        
        results = metadata.copy()
        
        # Adding calculated data to table
        results["anomaly_scores"] = raw_scores
        results["percentile_rank"] = percentiles
        results["risk_levels"] = risk_levels
        
        # Ranking highest risk levels first
        results.sort_values("percentile_rank", ascending=False, inplace=True)
        
        return results

### Loading UEBA Behavioral Matrix and Anomaly Scores:

In [3]:
# Loading UEBA-enhanced behavioral matrix
ueba_matrix = pd.read_csv(r"processed_datasets\ueba_dataset.csv", index_col=0)

In [4]:
# Extracting metadata for analyst-facing table
metadata = ueba_matrix[["user", "day"]]

In [5]:
anomaly_scores = np.load(r"isolation_forests\iforest_model_1\anomaly_scores.npy")

### Initializing and Building an Analyst-Ready Table:

In [6]:
interpreter = UEBAAnomalyInterpretor(
    high_risk_percentile=95,
    medium_risk_percentile=80
)

In [7]:
# Building the analyst-ready table
analyst_table = interpreter.build_analyst_table(metadata=metadata, raw_scores=anomaly_scores)

In [8]:
analyst_table

,user,day,anomaly_scores,percentile_rank,risk_levels
1159995,plj1771,2010-08-12,0.811560,100.000000,HIGH
1301274,sdh2394,2011-02-21,0.805815,99.999936,HIGH
47234,agf0742,2011-03-25,0.803365,99.999872,HIGH
47158,agf0742,2010-12-14,0.802822,99.999776,HIGH
1301253,sdh2394,2011-01-29,0.802822,99.999776,HIGH
...,...,...,...,...,...
312341,cma2616,2010-03-04,0.333255,0.000320,LOW
228314,cac0147,2011-05-20,0.333232,0.000256,LOW
1394917,tef3555,2010-09-22,0.333217,0.000192,LOW
1167445,ppt3162,2010-10-18,0.333187,0.000128,LOW


In [9]:
# Defining save path
save_path = r"static_dashboards"
os.makedirs(save_path, exist_ok=True)

In [10]:
# Saving the analyst table
analyst_table.to_csv(os.path.join(save_path, "table_1.csv"), index=False)